In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import math
from scipy import signal
from scipy.fftpack import fft
from scipy import ndimage
%matplotlib inline

In [2]:
import os
os.chdir('C:\\Users\\hydro_leo\\Documents\\GitHub\\BROJA_2PID')
from BROJA_2PID import pid, BROJA_2PID_Exception

In [2]:
def PIfunc(r, x, v, dt, window):
    negshift=window[0] # second
    posshift=window[1] # second
    shiftlen=(posshift-negshift)/dt+1
    timeshift=np.linspace(negshift,posshift,int(shiftlen))
    bitshift=np.linspace(negshift/dt,posshift/dt,int(shiftlen),dtype = 'int16')
    Redun =np.zeros(len(bitshift))
    MIxr=np.zeros(len(bitshift))
    MIvr=np.zeros(len(bitshift))
    MIxvR=np.zeros(len(bitshift))
    # shifted data
    # shift>0 => y shifted to positive side
    for i in range(len(bitshift)):
        xx=[]
        vv=[]
        rr=[]
        shift=bitshift[i]
        if shift>0:
            xx=x[shift:]
            vv=v[shift:]
            rr=r[:(-1*shift)]
        elif shift==0:
            xx=x
            vv=v
            rr=r
        elif shift<0:
            xx=x[:shift]
            vv=v[:shift]
            rr=r[(-1*shift):]
        #find weight of each states by 3D histogram 
        xedges = np.append(np.unique(xx),(max(xx)+1))
        vedges = np.append(np.unique(vv),(max(vv)+1))
        redges = np.append(np.unique(rr),(max(rr)+1))
        dat = np.concatenate((xx[:,np.newaxis], vv[:,np.newaxis],rr[:,np.newaxis]), axis=1)
        N, edges = np.histogramdd(dat, bins=(xedges, vedges, redges))
        #Calculate all kinds of probability and make sure the shape of them, 0 -> x, 1 -> v, 2 -> r
        px=(np.sum(N,axis=(1,2))/np.sum(N))[:, np.newaxis, np.newaxis]
        pv=(np.sum(N,axis=(0,2))/np.sum(N))[np.newaxis, :, np.newaxis]
        pr=(np.sum(N,axis=(0,1))/np.sum(N))[np.newaxis ,np.newaxis, :]
        pxv=(np.sum(N,axis=2)/np.sum(N))[:, :, np.newaxis]
        pxr=(np.sum(N,axis=1)/np.sum(N))[:, np.newaxis, :]
        pvr=(np.sum(N,axis=0)/np.sum(N))[np.newaxis, :, :]
        pxvr=(N/np.sum(N))
        
        MIxr[i]=np.nansum(pxr*np.log2(pxr/px/pr))/dt
        MIvr[i]=np.nansum(pvr*np.log2(pvr/pv/pr))/dt
        MIxvR[i]=np.nansum(pxvr*np.log2(pxvr/pxv/pr))/dt
        
        PI_xR = np.nansum(pxr*np.log2(pxr/px/pr), axis = (0,1))
        PI_vR = np.nansum(pvr*np.log2(pvr/pv/pr), axis = (0,1))
        Redun[i] = sum(np.minimum(PI_xR, PI_vR))/dt
    return timeshift, MIxr, MIvr, MIxvR, Redun

In [8]:
#calculate STA
xstate = np.random.randint(2, size=1000)
vstate = np.random.randint(2, size=1000)
rstate = vstate*2+xstate
dt = 1
window = [0,0] # second
timeshift, MIrx, MIrv, MIjoint, Redun = PIfunc(rstate, xstate, vstate, dt, window)
timeshift, A, MIxv, B, C = PIfunc(xstate, xstate, vstate, dt, window)
Ux = MIrx-Redun
Uv = MIrv-Redun
Syner = MIjoint-MIrx-MIrv+Redun

C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in multiply
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in multiply
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in multiply
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in multi

In [9]:
print(Syner, MIxv)
print(Ux)
print(Uv)
print(Redun)
print(MIjoint)

[0.97479838] [0.00046814]
[0.02389944]
[0.02454881]
[0.97526652]
[1.99851315]


In [6]:
# test_and_gate.py
from BROJA_2PID import pid, BROJA_2PID_Exception

# AND gate
andgate = dict()
andgate[ (0,0,0) ] = .25
andgate[ (0,0,1) ] = .25
andgate[ (0,1,2) ] = .25
andgate[ (1,1,3) ] = .25

# ECOS parameters 
parms = dict()
parms['max_iters'] = 10

print("Starting BROJA_2PID.pid() on AND gate.")
try:
  returndict = pid(andgate, cone_solver="ECOS", output=2, **parms)

  msg="""Shared information: {SI}
  Unique information in Y: {UIY}
  Unique information in Z: {UIZ}
  Synergistic information: {CI}
  Primal feasibility: {Num_err[0]}
  Dual feasibility: {Num_err[1]}
  Duality Gap: {Num_err[2]}"""
  print(msg.format(**returndict))
  
except BROJA_2PID_Exception:
  print("Cone Programming solver failed to find (near) optimal solution. Please report the input probability density function to abdullah.makkeh@gmail.com")

print("The End")

Starting BROJA_2PID.pid() on AND gate.
BROJA_2PID: Preparing Cone Program data...done.
BROJA_2PID: Starting solver.

ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +2e+01  8e-01  3e-01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  -1.441e+00  -9.177e-01  +5e+00  9e-01  1e-01  9e-01  2e-01  0.7833  9e-03   1  1  1 |  1  1
 2  -1.258e+00  -1.091e+00  +1e+00  3e-01  3e-02  3e-01  5e-02  0.7833  9e-03   1  1  1 |  1  1
 3  -4.328e-01  -3.968e-01  +2e-01  8e-02  7e-03  6e-02  1e-02  0.7833  1e-02   1  1  1 |  1  1
 4  -4.802e-02  -3.725e-02  +4e-02  2e-02  2e-03  2e-02  2e-03  0.7833  5e-02   1  1  1 |  2  1
 5  -2.207e-02  -1.742e-02  +2e-02  1e-02  9e-04  8e-03  9e-04  0.6266  1e-01   1  1  1 |  3  2
 6  -8.261e-03  -7.311e-03  +4e-03  2e-03  2e-04  2e-03  2e-04  0.7830  3e-04   1  1  1 |  0  1
 7  -1.638e-03  -1.423e-03